In [1]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
from scipy.stats import linregress
import numpy as np
import hvplot.pandas
import requests
import json
from pprint import pprint



#Upload from csv
data_set_path = "school_attendance.csv"
data_set_2_path = "all_students_df.csv"
zip_codes_path = "zip_codes.csv"
#population_set_path = "school_attendance.csv"

#Read data and results
data_set_2_df = pd.read_csv(data_set_2_path)
zip_codes_df = pd.read_csv(zip_codes_path, converters={"Zip Code":str})
data_set_df = pd.read_csv(data_set_path)

#population_df = pd.read_csv(population_set_path)
data_set_df.head(30)

,District code,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Reporting period,Date update
0,00000CT,Connecticut,NaN,All Students,500285,0.9169,496092.0,0.9294,508346.0,0.9479,June 2022,07/22/2022
1,00000CT,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735.0,0.8155,3916.0,0.8884,June 2022,07/22/2022
2,00000CT,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487.0,0.8946,80365.0,0.9277,June 2022,07/22/2022
3,00000CT,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225.0,0.8861,193706.0,0.9314,June 2022,07/22/2022
4,00000CT,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886.0,0.9299,27507.0,0.9518,June 2022,07/22/2022
5,00000CT,Connecticut,Free/Reduced Lunch,Free/Reduced Price Meal Eligible,198889,0.8901,207111.0,0.8927,221213.0,0.9340,June 2022,07/22/2022
6,00000CT,Connecticut,English Learners,English Learners,43571,0.8976,40619.0,0.8948,45413.0,0.9389,June 2022,07/22/2022
7,00000CT,Connecticut,Race/Ethnicity,All other races,48700,0.9314,47339.0,0.9483,47260.0,0.9559,June 2022,07/22/2022
8,00000CT,Connecticut,Race/Ethnicity,Black or African American,63099,0.8941,62267.0,0.8931,64200.0,0.9401,June 2022,07/22/2022
9,00000CT,Connecticut,Race/Ethnicity,Hispanic/Latino of any race,146298,0.8935,138260.0,0.8975,136953.0,0.9362,June 2022,07/22/2022


In [19]:
#Attempt to clean out populations
all_students_df = data_set_df.loc[(data_set_df["Student group"]=="All Students")]
all_students_df = all_students_df[["District name", "2021-2022 attendance rate - year to date"]]
all_students_df["District name"] = all_students_df["District name"].str.upper()

#Drop Connecticut
all_students_df = all_students_df.drop(0)
all_students_df

,District name,2021-2022 attendance rate - year to date
13,ANDOVER SCHOOL DISTRICT,0.9386
19,ANSONIA SCHOOL DISTRICT,0.9045
31,ASHFORD SCHOOL DISTRICT,0.9309
40,AVON SCHOOL DISTRICT,0.9457
52,BARKHAMSTED SCHOOL DISTRICT,0.9305
...,...,...
1972,UNIFIED SCHOOL DISTRICT #2,0.9202
1977,CONNECTICUT TECHNICAL EDUCATION AND CAREER SYSTEM,0.8985
1990,NORWICH FREE ACADEMY DISTRICT,0.8940
2002,THE GILBERT SCHOOL DISTRICT,0.8901


In [20]:
#Rename columns
zip_codes_df = zip_codes_df.rename(columns={"District Name": "District name"})
zip_codes_df = zip_codes_df[["District name", "Zip Code"]]
zip_codes_df.head()

,District name,Zip Code
0,ACHIEVEMENT FIRST HARTFORD AC,06112
1,AMISTAD ACADEMY DISTRICT,06513
2,ANDOVER SCHOOL DISTRICT,06232
3,ANSONIA SCHOOL DISTRICT,06401
4,AREA COOPERATIVE EDUCATIONAL,06473


In [21]:
 #Merge Files
merged_df = pd.merge(all_students_df, zip_codes_df, on="District name", how = "left")
#merged_3_df = merged_df[["District name", "Zip Code"]]
merged_df = merged_df.dropna()
merged_df

,District name,2021-2022 attendance rate - year to date,Zip Code
0,ANDOVER SCHOOL DISTRICT,0.9386,06232
1,ANSONIA SCHOOL DISTRICT,0.9045,06401
2,ASHFORD SCHOOL DISTRICT,0.9309,06278
3,AVON SCHOOL DISTRICT,0.9457,06001
4,BARKHAMSTED SCHOOL DISTRICT,0.9305,06063
...,...,...,...
178,THE BRIDGE ACADEMY DISTRICT,0.9387,06608
180,EXPLORATIONS DISTRICT,0.8612,06098
181,AMISTAD ACADEMY DISTRICT,0.8832,06513
194,UNIFIED SCHOOL DISTRICT #1,0.9564,06109


In [22]:
# Import API key
from config3 import geoapify_key

In [23]:
#Set up base API URL
zip_code = "06473"
country = "United States"
url = f"https://api.geoapify.com/v1/geocode/search?text={zip_code}&format=json&apiKey={geoapify_key}"
geo_data = requests.get(url).json()

pprint(geo_data)


{'query': {'parsed': {'expected_type': 'unknown', 'postcode': '06473'},
           'text': '06473'},
 'results': [{'address_line1': 'North Haven',
              'address_line2': 'North Haven, CT 06473, United States of '
                               'America',
              'bbox': {'lat1': 41.221443578114,
                       'lat2': 41.541443578114,
                       'lon1': -73.019669765131,
                       'lon2': -72.699669765131},
              'city': 'North Haven',
              'country': 'United States',
              'country_code': 'us',
              'county': 'South Central Connecticut Planning Region',
              'datasource': {'attribution': '© OpenStreetMap contributors',
                             'license': 'Open Database License',
                             'sourcename': 'openstreetmap',
                             'url': 'https://www.openstreetmap.org/copyright'},
              'formatted': 'North Haven, CT 06473, United States of America',

In [25]:
zip_code = []
districts = []
for item in merged_df["Zip Code"]:
    zip_code.append(item)
for district in merged_df["2021-2022 attendance rate - year to date"]:
    districts.append(district)
districts

[0.9386,
 0.9045,
 0.9309,
 0.9457,
 0.9305,
 0.9341,
 0.9489,
 0.9375,
 0.9293,
 0.9433,
 0.9433,
 0.9268,
 0.9008,
 0.9251,
 0.9368,
 0.9204,
 0.9431,
 0.9148,
 0.9356,
 0.9293,
 0.9466,
 0.9563,
 0.939,
 0.9318,
 0.9241,
 0.9496,
 0.9384,
 0.9295,
 0.9445,
 0.9166,
 0.9553,
 0.9565,
 0.8979,
 0.9387,
 0.9301,
 0.9391,
 0.9413,
 0.91,
 0.92,
 0.9481,
 0.9499,
 0.9003,
 0.943,
 0.9211,
 0.9496,
 0.9385,
 0.9442,
 0.9376,
 0.9509,
 0.9449,
 0.9418,
 0.9234,
 0.9144,
 0.9372,
 0.8962,
 0.9111,
 0.8564,
 0.9383,
 0.9527,
 0.9455,
 0.9093,
 0.9386,
 0.9117,
 0.9206,
 0.9299,
 0.9593,
 0.8854,
 0.9356,
 0.9462,
 0.9112,
 0.8941,
 0.9227,
 0.9485,
 0.9225,
 0.9184,
 0.8671,
 0.9566,
 0.9288,
 0.9432,
 0.8383,
 0.9413,
 0.8868,
 0.9183,
 0.9465,
 0.9372,
 0.9407,
 0.9464,
 0.9092,
 0.9235,
 0.9466,
 0.9435,
 0.9413,
 0.9151,
 0.9241,
 0.8993,
 0.9371,
 0.9386,
 0.9136,
 0.8996,
 0.9505,
 0.9446,
 0.948,
 0.9435,
 0.9395,
 0.9128,
 0.9296,
 0.9281,
 0.9369,
 0.94,
 0.9414,
 0.9478,
 0.9409,
 

In [34]:
district_data = []
country_code = "us"

i=0
for code in zip_code:
    try:
        #zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}&format=json&apiKey={geoapify_key}"
        zip_code_url = f"https://api.geoapify.com/v1/geocode/search?text={code}%20{country_code}&format=json&apiKey={geoapify_key}"
        zip_code_requests = requests.get(zip_code_url).json()
        print(f"Zip Code URL: {zip_code_url}")
    #Find Lat and Long
        zip_lat = zip_code_requests['results'][0]['lat']
        zip_lng = zip_code_requests['results'][0]['lon']
        zip_code_1 = zip_code_requests["results"][0]["postcode"]
        
       
     # Append the City information into city_data list
        district_data.append({"Zip Code": zip_code_1, 
                          "Lat": zip_lat, 
                          "Lng": zip_lng, 
                            "Population": districts[i]*25 
                          })
    except:
        print(f"not working")
    i=i+1        

Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06232%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06401%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06278%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06001%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06063%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06037%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06524%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06801%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06040%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06268%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06447%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06450%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06457%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06460%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06468%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06370%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06498%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06107%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06516%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06883%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06880%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06109%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06279%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e
Zip Code URL: https://api.geoapify.com/v1/geocode/search?text=06897%20us&format=json&apiKey=c967a2b03b354238ac3aa60aa86eb97e


In [35]:
district_data_df = pd.DataFrame(district_data)
district_data_df

,Zip Code,Lat,Lng,Population
0,06232,41.732991,-72.375790,23.4650
1,06401,41.342445,-73.072524,22.6125
2,06278,41.890680,-72.171084,23.2725
3,06001,41.791284,-72.866740,23.6425
4,06063,41.918415,-72.977078,23.2625
...,...,...,...,...
165,06608,41.189517,-73.180982,23.4675
166,06098,41.927121,-73.078201,21.5300
167,06513,41.316445,-72.875431,22.0800
168,06109,41.700966,-72.676233,23.9100


In [36]:
map_plot_1 = district_data_df.hvplot.points(
    "Lng",
    "Lat",
    s="Population",
    geo = True,
    tiles = "OSM",
)
hvplot.save(map_plot_1, "attendancemap.png")
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Population)

*** Analysis: This map shows that there is a higher concentration of students around major cities, specifically Hartford, New Haven, and Bridgeport. The more rural districts have less students.***